In [1]:
!pip install geojson
!pip install rasterio

# !pip install rioxarray


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 15.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from osgeo import gdal
import numpy as np
import statistics
import rasterio
from rasterio import features
import matplotlib.pyplot as plt
from rasterio.plot import plotting_extent
import geopandas as gpd
import fiona
from rasterio.crs import CRS
from rasterio.mask import mask
import pandas as pd
import json
import geojson

import keras
from tensorflow.keras import Input
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from keras.layers import LeakyReLU

from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import train_test_split

In [4]:
dataset1 = gdal.Open(r'/content/drive/MyDrive/WQA_data/DLdata/T_09-07-2023.tif')
dataset2 = gdal.Open(r'/content/drive/MyDrive/WQA_data/DLdata/T_20-04-2023.tif')

dataframe = pd.read_csv('/content/drive/MyDrive/WQA_data/AvantiCombined_NSR.csv')


In [5]:
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']
wavelengths = [442.7,	492.4,559.8,664.6,704.1,740.5,782.8,832.8,864.7,945.1,1373.5,1613.7,2202.4]
parameters = ["pH", "Salinity", "Ammonia"]

In [6]:
with open(f'/content/drive/MyDrive/WQA_data/CroppedPond.geojson') as f:
    gj = geojson.load(f)

files = [r"/content/drive/MyDrive/WQA_data/DLdata/T_09-07-2023.tif"\
         ,r"/content/drive/MyDrive/WQA_data/DLdata/T_20-04-2023.tif"\
        ]



def indexDataGen(image, index):
  if index == 'NDWI' :
    arr = (image[:,:,2] -image[:,:,7] )/(image[:,:,2] +image[:,:,7] )
    arr[np.isnan(arr)] = 0
    arr[np.isinf(arr)] = 0
    return arr
  # if index == 'EWI' :return  (image["B3"] -image["B8"] -image["B11"])/(image["B3"] +image["B8"]+image["B11"])
  # if index == 'NWI' :return  (image["B1"] -(image["B8"] +image["B11"]+image["B12"]))/(image["B1"] +image["B8"]+image["B11"]+image["B12"])

  if index == 'NDVI' :
        arr = (image[:,:,7] -image[:,:,5] )/(image[:,:,7] +image[:,:,5] )
        arr[np.isnan(arr)] = 0
        arr[np.isinf(arr)] = 0
        return arr
  # if index == 'norm_B9_B11' :return  (image["B9"] -image["B11"] )/(image["B9"] +image["B11"] )

  # if index == 'AWEI' :return  4 * (image['B3'] - image['B12'] ) - (2.5 * image['B8']  + 2.75 * image['B11'])

bandindex = 'NDVI'

imageData = {}
dates = ['2023-07-09','2023-04-20']
Tresults = []
for index,f in zip( dates,files):
  results = {}
  for i in gj['features']:
      name = (i['properties']['Pond'])
      with rasterio.open(f) as src:
        out_image, out_transform = rasterio.mask.mask(src, [i['geometry']], crop=True)
        reshaped_image = np.transpose(out_image, (1, 2, 0))
        # if name in ['T_B3', 'T_B4', 'T_B5', 'T_B2','T_C8',"T_A1"]:
          # print(name)
          # plt.figure()
        array = indexDataGen(reshaped_image,bandindex)
        imageData.update({f'{index}_{name}':array})

        # nir = reshaped_image[:,:,7]
    # return imageData


<ipython-input-6-30ad957871a7>:20: RuntimeWarning: invalid value encountered in true_divide
  arr = (image[:,:,7] -image[:,:,5] )/(image[:,:,7] +image[:,:,5] )
<ipython-input-6-30ad957871a7>:20: RuntimeWarning: invalid value encountered in true_divide
  arr = (image[:,:,7] -image[:,:,5] )/(image[:,:,7] +image[:,:,5] )
<ipython-input-6-30ad957871a7>:20: RuntimeWarning: invalid value encountered in true_divide
  arr = (image[:,:,7] -image[:,:,5] )/(image[:,:,7] +image[:,:,5] )
<ipython-input-6-30ad957871a7>:20: RuntimeWarning: invalid value encountered in true_divide
  arr = (image[:,:,7] -image[:,:,5] )/(image[:,:,7] +image[:,:,5] )
<ipython-input-6-30ad957871a7>:20: RuntimeWarning: invalid value encountered in true_divide
  arr = (image[:,:,7] -image[:,:,5] )/(image[:,:,7] +image[:,:,5] )
<ipython-input-6-30ad957871a7>:20: RuntimeWarning: invalid value encountered in true_divide
  arr = (image[:,:,7] -image[:,:,5] )/(image[:,:,7] +image[:,:,5] )
<ipython-input-6-30ad957871a7>:20: Runti

In [7]:
  # imagearray  = list(imageData.values())
# list(imageData.values())

In [8]:
tempdf = dataframe[dataframe['Date of Image']=='2023-07-09']
temparray = []
for index, row in tempdf.iterrows():
  id = row['PondID'].split('_')
  # print(id)
  if id[0] == 'TUPILIPALEM':name = 'T'
  temparray.append(imageData[f'2023-07-09_{name}_{id[1]}'])
print(tempdf.shape)

tempdf2 = dataframe[dataframe['Date of Image']=='2023-04-20'][dataframe['PondID'] !='KOKKUPADU']
print(tempdf.shape)
# temparray = []
for index, row in tempdf2.iterrows():
  id = row['PondID'].split('_')[1]
  temparray.append(imageData[f'2023-04-20_T_{id}'])

<ipython-input-8-0e412135a16a>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tempdf2 = dataframe[dataframe['Date of Image']=='2023-04-20'][dataframe['PondID'] !='KOKKUPADU']


(15, 26)
(15, 26)


In [9]:
# df = df[df.line_race != 0]
filterdf = pd.concat([tempdf,tempdf2])
len(filterdf)

32

In [10]:
len(temparray)

32

In [11]:
gt = filterdf[parameters]
gt.head()

,pH,Salinity,Ammonia
131,8.0,35,0.04
132,8.5,40,0.08
133,8.1,40,0.08
134,8.2,40,0.05
135,7.9,36,0.08


In [12]:
NewTempArray = []
for i  in temparray:
  # print(i.shape)
  # i = np.pad(i, ((128- i.shape[0], 199-i.shape[1]), (199-i.shape[1], 128- i.shape()[0])), 'constant')
  # print(i.shape)
  newx = 128- i.shape[0]
  newy = 199-i.shape[1]
  # print(i, np.zeros(newx,))
  updatedTempArray = np.append(i, np.zeros((newx,i.shape[1])), axis=0)
  updatedTempArray = np.append(updatedTempArray, np.zeros((updatedTempArray.shape[0],newy)), axis=1)
  # NewTempArray = np.append(NewTempArray,[updatedTempArray])
  NewTempArray.append(updatedTempArray)
  # print(updatedTempArray.shape)
    # i.reshape((121,147))

In [13]:
NewTempArray = np.array(NewTempArray)

In [14]:

np.any(np.isnan(NewTempArray))

False

In [15]:
np.any(np.isinf(NewTempArray))

False

In [16]:
SelectPara = ['pH',"Salinity",'Ammonia']
X, y = NewTempArray, np.array(gt[SelectPara],dtype=np.float32)
# X, y = X.astype('float'), y.astype('float')
n_outputs = len(SelectPara)
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

# X_train = X_train.reshape(-1, 128, 199, 1)

# X_test = X_test.reshape(-1, 128, 199, 1)

# define the keras model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(128,199,1),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2, 2),padding='same'))
model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(n_outputs))
model.compile(loss='mse', optimizer='adam')
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=150, batch_size=10, verbose=2)
# evaluate on test set
yhat = model.predict(X_test)


Epoch 1/150
3/3 - 6s - loss: 428.4261 - 6s/epoch - 2s/step
Epoch 2/150
3/3 - 3s - loss: 440.4745 - 3s/epoch - 845ms/step
Epoch 3/150
3/3 - 3s - loss: 229.8938 - 3s/epoch - 865ms/step
Epoch 4/150
3/3 - 4s - loss: 156.9517 - 4s/epoch - 1s/step
Epoch 5/150
3/3 - 3s - loss: 72.9222 - 3s/epoch - 1s/step
Epoch 6/150
3/3 - 2s - loss: 50.7851 - 2s/epoch - 680ms/step
Epoch 7/150
3/3 - 1s - loss: 31.1609 - 1s/epoch - 486ms/step
Epoch 8/150
3/3 - 1s - loss: 21.9140 - 1s/epoch - 439ms/step
Epoch 9/150
3/3 - 1s - loss: 14.7013 - 1s/epoch - 433ms/step
Epoch 10/150
3/3 - 1s - loss: 22.9066 - 1s/epoch - 418ms/step
Epoch 11/150
3/3 - 2s - loss: 7.0541 - 2s/epoch - 605ms/step
Epoch 12/150
3/3 - 3s - loss: 6.0990 - 3s/epoch - 852ms/step
Epoch 13/150
3/3 - 2s - loss: 6.2780 - 2s/epoch - 575ms/step
Epoch 14/150
3/3 - 1s - loss: 4.6296 - 1s/epoch - 444ms/step
Epoch 15/150
3/3 - 1s - loss: 1.9546 - 1s/epoch - 484ms/step
Epoch 16/150
3/3 - 1s - loss: 2.5134 - 1s/epoch - 469ms/step
Epoch 17/150
3/3 - 1s - loss

In [17]:
parara = 2
error = mean_absolute_error(y_test[:,parara], yhat[:,parara])
print('MAE: %.3f' % error)
RMSEerror = mean_squared_error(y_test[:,parara], yhat[:,parara],squared=False)
print('RMSE: %.3f' % RMSEerror)

r2 = r2_score(y_test[:,parara], yhat[:,parara])
print('R2: %.3f' % r2)

MAE: 0.321
RMSE: 0.352
R2: -1663.298


In [18]:
y_test, yhat

(array([[ 8.4 , 30.  ,  0.04],
        [ 8.2 , 40.  ,  0.05],
        [ 8.4 , 32.  ,  0.05],
        [ 8.3 , 30.  ,  0.05],
        [ 8.1 , 31.  ,  0.04],
        [ 8.2 , 31.  ,  0.05],
        [ 8.2 , 36.  ,  0.04],
        [ 8.2 , 27.  ,  0.04],
        [ 7.6 , 28.  ,  0.07],
        [ 7.7 , 39.  ,  0.04],
        [ 8.3 , 30.  ,  0.05]], dtype=float32),
 array([[ 4.97776890e+00,  2.03604088e+01,  3.62184979e-02],
        [ 8.21236134e+00,  3.91726036e+01, -2.84581423e-01],
        [ 8.08097076e+00,  3.77292976e+01, -4.25179005e-01],
        [ 7.99285126e+00,  3.17533798e+01, -3.43253165e-01],
        [ 8.17736053e+00,  3.38852196e+01, -1.19984195e-01],
        [ 7.88900995e+00,  3.30067215e+01, -3.84954184e-01],
        [ 7.97075987e+00,  3.75266724e+01, -2.67084837e-01],
        [ 7.08007622e+00,  2.67926121e+01, -4.01344448e-01],
        [ 7.20184946e+00,  3.17033615e+01, -9.27824229e-02],
        [ 8.27796364e+00,  3.99122200e+01, -4.23627526e-01],
        [ 8.27296352e+00,  3.538

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 199, 32)      320       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 128, 199, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 100, 32)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 100, 64)       18496     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 64, 100, 64)       0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 50, 64)       0         
 2D)                                                    